Städte:
- Kopenhagen
- Hamburg
- Paris
- Amsterdam
- Stockholm
- Budapest
- Lissabon
- Mailand (Milano)
– Barcelona

Countries:
- Spain, Italy, Hungary, France, Germany, Netherlands, Portugal, Sweden, Denmark

In [12]:
import os
import re
import glob
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd

# Paths (CSV-only workflow)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
RAW_DIR = os.path.join(BASE_DIR, "data", "raw")
OUTPUT_PROCESSED_DIR = os.path.join(BASE_DIR, "output", "processed")
OUTPUT_METRICS_DIR = os.path.join(BASE_DIR, "output", "metrics")

os.makedirs(OUTPUT_PROCESSED_DIR, exist_ok=True)
os.makedirs(OUTPUT_METRICS_DIR, exist_ok=True)



In [20]:

def sanitize_identifier(path: str) -> str:
    base = os.path.splitext(os.path.basename(path))[0]
    s = re.sub(r"[^0-9A-Za-z_]+", "_", base)
    if re.match(r"^\d", s or ""):
        s = f"_{s}"
    return (s or "dataset").lower()


def list_csv_files(raw_dir: str = RAW_DIR) -> List[str]:
    return sorted(glob.glob(os.path.join(raw_dir, "*.csv")))


def split_numeric_and_categorical(df: pd.DataFrame) -> Tuple[List[str], List[str]]:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = [c for c in df.columns if c not in numeric_cols]
    return numeric_cols, categorical_cols


def compute_numeric_stats(series: pd.Series) -> Dict[str, Optional[float]]:
    ser = pd.to_numeric(series, errors="coerce")
    row_count = int(len(ser))
    non_null_count = int(ser.notna().sum())
    null_count = int(row_count - non_null_count)
    if non_null_count == 0:
        return {
            "column": series.name,
            "row_count": row_count,
            "non_null_count": non_null_count,
            "null_count": null_count,
            "mean": None,
            "stddev": None,
            "min": None,
            "q1": None,
            "median": None,
            "q3": None,
            "max": None,
            "iqr": None,
            "lower_bound": None,
            "upper_bound": None,
            "outlier_count": None,
        }
    q1 = float(np.nanpercentile(ser, 25))
    median = float(np.nanpercentile(ser, 50))
    q3 = float(np.nanpercentile(ser, 75))
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outlier_mask = (ser < lower_bound) | (ser > upper_bound)
    return {
        "column": series.name,
        "row_count": row_count,
        "non_null_count": non_null_count,
        "null_count": null_count,
        "mean": float(np.nanmean(ser)),
        "stddev": float(np.nanstd(ser, ddof=1)) if non_null_count > 1 else 0.0,
        "min": float(np.nanmin(ser)),
        "q1": q1,
        "median": median,
        "q3": q3,
        "max": float(np.nanmax(ser)),
        "iqr": float(iqr),
        "lower_bound": float(lower_bound),
        "upper_bound": float(upper_bound),
        "outlier_count": int(outlier_mask.sum()),
    }


def compute_categorical_stats(series: pd.Series) -> Dict[str, Optional[float]]:
    ser = series.copy()
    row_count = int(len(ser))
    non_null_count = int(ser.notna().sum())
    null_count = int(row_count - non_null_count)
    distinct_count = int(ser.nunique(dropna=True))
    if non_null_count > 0:
        mode_val = ser.mode(dropna=True)
        mode_value = mode_val.iloc[0] if not mode_val.empty else "Unknown"
        mode_freq = int((ser == mode_value).sum()) if mode_value != "Unknown" else 0
    else:
        mode_value, mode_freq = "Unknown", 0
    return {
        "column": series.name,
        "row_count": row_count,
        "non_null_count": non_null_count,
        "null_count": null_count,
        "distinct_count": distinct_count,
        "mode": mode_value,
        "mode_freq": mode_freq,
    }


def add_derived_columns(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy()
    numeric_cols, categorical_cols = split_numeric_and_categorical(df_out)

    for col in numeric_cols:
        ser = pd.to_numeric(df_out[col], errors="coerce")
        if ser.notna().any():
            q1 = np.nanpercentile(ser, 25)
            q3 = np.nanpercentile(ser, 75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            median = np.nanmedian(ser)
            cleaned = ser.fillna(median).clip(lower=lower_bound, upper=upper_bound)
        else:
            cleaned = ser.fillna(0.0)
        df_out[f"{col}_clean"] = cleaned

    for col in categorical_cols:
        ser = df_out[col]
        if ser.notna().any():
            mode_series = ser.mode(dropna=True)
            fill_value = mode_series.iloc[0] if not mode_series.empty else "Unknown"
        else:
            fill_value = "Unknown"
        df_out[f"{col}_filled"] = ser.fillna(fill_value)

    if "TIME_PERIOD" in df_out.columns and "year" not in df_out.columns:
        year = pd.to_datetime(df_out["TIME_PERIOD"], errors="coerce").dt.year
        df_out["year"] = year

    return df_out


def profile_dataframe(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    numeric_cols, categorical_cols = split_numeric_and_categorical(df)
    numeric_stats = [compute_numeric_stats(df[c]) for c in numeric_cols]
    categorical_stats = [compute_categorical_stats(df[c]) for c in categorical_cols]
    numeric_df = pd.DataFrame(numeric_stats)
    categorical_df = pd.DataFrame(categorical_stats)
    return numeric_df, categorical_df


def process_csv_file(csv_path: str) -> None:
    dataset = sanitize_identifier(csv_path)
    df = pd.read_csv(csv_path, low_memory=False)
    processed = add_derived_columns(df)
    numeric_df, categorical_df = profile_dataframe(df)

    processed_out = os.path.join(OUTPUT_PROCESSED_DIR, f"{dataset}.csv")
    metrics_num_out = os.path.join(OUTPUT_METRICS_DIR, f"{dataset}_numeric_stats.csv")
    metrics_cat_out = os.path.join(OUTPUT_METRICS_DIR, f"{dataset}_categorical_stats.csv")

    processed.to_csv(processed_out, index=False)
    numeric_df.to_csv(metrics_num_out, index=False)
    categorical_df.to_csv(metrics_cat_out, index=False)



In [14]:

from IPython.display import display

# List available CSVs
csv_files = list_csv_files(RAW_DIR)
print(f"Found {len(csv_files)} CSVs in {RAW_DIR}.")
display(pd.DataFrame({
    "index": list(range(len(csv_files))),
    "filename": [os.path.basename(p) for p in csv_files],
}))

# Choose which CSV to analyze (edit this line)
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "hlth_rs_bds1_page_linear_2_0.csv")

# Analyze selected CSV and show results inline
print(f"\nAnalyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df)
numeric_df, categorical_df = profile_dataframe(df)

print("Raw dataframe shape:", df.shape)
print("Processed dataframe shape:", processed.shape)

# Preview processed data
display(processed.head(10))

# Show numeric column stats (sorted by non-null count)
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Show categorical column stats (sorted by distinct count)
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns:
# OBS_VALUE: Betten pro 100k einwohner, landes und jahresbezogen

Found 15 CSVs in /Users/felixwegener/freelancing/data-analytics-project/data/raw.


,index,filename
0,0,hlth_rs_bds1_page_linear_2_0.csv
1,1,isoc_ciegi_ac_page_linear_2_0.csv
2,2,sdg_11_60_page_linear_2_0.csv
3,3,sdg_17_60_page_linear_2_0.csv
4,4,tour_occ_nim_page_linear_2_0.csv
5,5,trng_lfse_01_page_linear_2_0.csv
6,6,urb_ceduc_page_linear_2_0.csv
7,7,urb_clma_page_linear_2_0(1).csv
8,8,urb_clma_page_linear_2_0.csv
9,9,urb_cpop1_page_linear_2_0 2.csv



Analyzing: hlth_rs_bds1_page_linear_2_0.csv
Raw dataframe shape: (337, 21)
Processed dataframe shape: (337, 43)


,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,freq,Time frequency,unit,Unit of measure,facility,Health facility,hlthcare,...,Unit of measure_filled,facility_filled,Health facility_filled,hlthcare_filled,Health care_filled,geo_filled,Geopolitical entity (reporting)_filled,OBS_FLAG_filled,Observation status (Flag) V2 structure_filled,year
0,dataflow,ESTAT:HLTH_RS_BDS1(1.0),Hospital beds by function and type of care,A,Annual,P_HTHAB,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,...,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,Total,AT,Austria,d,definition differs (see metadata),1970
1,dataflow,ESTAT:HLTH_RS_BDS1(1.0),Hospital beds by function and type of care,A,Annual,P_HTHAB,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,...,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,Total,AT,Austria,d,definition differs (see metadata),1970
2,dataflow,ESTAT:HLTH_RS_BDS1(1.0),Hospital beds by function and type of care,A,Annual,P_HTHAB,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,...,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,Total,AT,Austria,d,definition differs (see metadata),1970
3,dataflow,ESTAT:HLTH_RS_BDS1(1.0),Hospital beds by function and type of care,A,Annual,P_HTHAB,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,...,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,Total,AT,Austria,d,definition differs (see metadata),1970
4,dataflow,ESTAT:HLTH_RS_BDS1(1.0),Hospital beds by function and type of care,A,Annual,P_HTHAB,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,...,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,Total,AT,Austria,d,definition differs (see metadata),1970
5,dataflow,ESTAT:HLTH_RS_BDS1(1.0),Hospital beds by function and type of care,A,Annual,P_HTHAB,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,...,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,Total,AT,Austria,d,definition differs (see metadata),1970
6,dataflow,ESTAT:HLTH_RS_BDS1(1.0),Hospital beds by function and type of care,A,Annual,P_HTHAB,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,...,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,Total,AT,Austria,d,definition differs (see metadata),1970
7,dataflow,ESTAT:HLTH_RS_BDS1(1.0),Hospital beds by function and type of care,A,Annual,P_HTHAB,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,...,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,Total,AT,Austria,d,definition differs (see metadata),1970
8,dataflow,ESTAT:HLTH_RS_BDS1(1.0),Hospital beds by function and type of care,A,Annual,P_HTHAB,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,...,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,Total,AT,Austria,d,definition differs (see metadata),1970
9,dataflow,ESTAT:HLTH_RS_BDS1(1.0),Hospital beds by function and type of care,A,Annual,P_HTHAB,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,...,Per hundred thousand inhabitants,HBEDT,Available beds in hospitals (HP.1),TOTAL,Total,BE,Belgium,d,definition differs (see metadata),1970


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,OBS_VALUE,337,337,0,471.226647,169.334570,101.62,318.25,446.33,587.48,864.39,269.23,-85.595,991.325,0.0
1,TIME_PERIOD,337,337,0,2018.991098,2.592901,2015.00,2017.00,2019.00,2021.00,2024.00,4.00,2011.000,2027.000,0.0
2,CONF_STATUS,337,0,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Confidentiality status (flag),337,0,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Observation value,337,0,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Time,337,0,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (reporting),337,337,0,38,Belgium,10
1,geo,337,337,0,38,BE,10
2,OBS_FLAG,337,39,298,6,d,14
3,Observation status (Flag) V2 structure,337,39,298,6,definition differs (see metadata),14
4,Health care,337,337,0,1,Total,337
5,Health facility,337,337,0,1,Available beds in hospitals (HP.1),337
6,STRUCTURE,337,337,0,1,dataflow,337
7,STRUCTURE_ID,337,337,0,1,ESTAT:HLTH_RS_BDS1(1.0),337
8,STRUCTURE_NAME,337,337,0,1,Hospital beds by function and type of care,337
9,Time frequency,337,337,0,1,Annual,337


In [15]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "REPLACE_WITH_FILENAME.csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))



Analyzing: REPLACE_WITH_FILENAME.csv


FileNotFoundError: [Errno 2] No such file or directory: '/Users/felixwegener/freelancing/data-analytics-project/data/raw/REPLACE_WITH_FILENAME.csv'

In [16]:
from IPython.display import display

# Choose which CSV to analyze (edit this line)
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "isoc_ciegi_ac_page_linear_2_0.csv")

print(f"\nAnalyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df)
numeric_df, categorical_df = profile_dataframe(df)

print("Raw dataframe shape:", df.shape)
print("Processed dataframe shape:", processed.shape)

# Preview processed data
# display(processed.head(5))

# Show numeric column stats (sorted by non-null count)
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Show categorical column stats (sorted by distinct count)
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns:
# E-government activities of individuals via websites
# only 2020 and 2021 data
# OBS_VALUE: Percentage of individuals of the country using e-government services


Analyzing: isoc_ciegi_ac_page_linear_2_0.csv
Raw dataframe shape: (76, 21)
Processed dataframe shape: (76, 43)


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,OBS_VALUE,76,76,0,58.797763,22.317688,13.43,44.275,60.58,76.575,94.15,32.3,-4.175,125.025,0.0
1,TIME_PERIOD,76,76,0,2020.500000,0.503322,2020.00,2020.000,2020.50,2021.000,2021.00,1.0,2018.500,2022.500,0.0
2,CONF_STATUS,76,0,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Confidentiality status (flag),76,0,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Observation value,76,0,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Time,76,0,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (reporting),76,76,0,40,Albania,2
1,geo,76,76,0,40,AL,2
2,OBS_FLAG,76,8,68,3,b,5
3,Observation status (Flag) V2 structure,76,8,68,3,break in time series,5
4,Individual type,76,76,0,1,All individuals,76
5,Information society indicator,76,76,0,1,Internet use: interaction with public authorit...,76
6,STRUCTURE,76,76,0,1,dataflow,76
7,STRUCTURE_ID,76,76,0,1,ESTAT:ISOC_CIEGI_AC(1.0),76
8,STRUCTURE_NAME,76,76,0,1,E-government activities of individuals via web...,76
9,Time frequency,76,76,0,1,Annual,76


In [17]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "sdg_11_60_page_linear_2_0.csv")
NUMERIC_COLUMNS = ["OBS_VALUE", "TIME_PERIOD"]  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns:
# OBS_VALUE: Recycling rate of municipal waste in percentage
# TIME_PERIOD: Year


Analyzing: sdg_11_60_page_linear_2_0.csv


TypeError: add_derived_columns() got an unexpected keyword argument 'numeric_include'

In [ ]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "sdg_17_60_page_linear_2_0.csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: High-speed internet coverage, by type of area, percentage of households with access
# Fixed very high capacity networks (VHCN)
# TIME_PERIOD: Year

Analyzing: sdg_17_60_page_linear_2_0.csv
Raw dataframe shape: (192, 21) Processed shape: (192, 43)
Numeric columns used: ['TIME_PERIOD', 'Time', 'OBS_VALUE', 'Observation value', 'OBS_FLAG', 'Observation status (Flag) V2 structure', 'CONF_STATUS', 'Confidentiality status (flag)']
Categorical columns used: ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'unit', 'Unit of measure', 'inet_tec', 'Internet technologies', 'terrtypo', 'Territorial typology', 'geo', 'Geopolitical entity (reporting)']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,OBS_VALUE,192,192,0,71.922917,21.069275,7.1,61.725,75.85,88.775,100.0,27.05,21.15,129.35,7.0
1,TIME_PERIOD,192,192,0,2021.500000,1.712290,2019.0,2020.000,2021.50,2023.000,2024.0,3.00,2015.50,2027.50,0.0
2,CONF_STATUS,192,0,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Confidentiality status (flag),192,0,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OBS_FLAG,192,0,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Observation status (Flag) V2 structure,192,0,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Observation value,192,0,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Time,192,0,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (reporting),192,192,0,32,Austria,6
1,geo,192,192,0,32,AT,6
2,Internet technologies,192,192,0,1,Fixed very high capacity networks (VHCN),192
3,STRUCTURE,192,192,0,1,dataflow,192
4,STRUCTURE_ID,192,192,0,1,ESTAT:SDG_17_60(1.0),192
5,STRUCTURE_NAME,192,192,0,1,"High-speed internet coverage, by type of area",192
6,Territorial typology,192,192,0,1,Total,192
7,Time frequency,192,192,0,1,Annual,192
8,Unit of measure,192,192,0,1,Percentage of households,192
9,freq,192,192,0,1,A,192


In [ ]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "tour_occ_nim_page_linear_2_0.csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: Nights spent at tourist accommodation establishments - monthly data
# Fixed very high capacity networks (VHCN)
# TIME_PERIOD: monthly 
# 2024-11 -> 2025-08

Analyzing: tour_occ_nim_page_linear_2_0.csv
Raw dataframe shape: (385, 21) Processed shape: (385, 43)
Numeric columns used: ['Time', 'OBS_VALUE', 'Observation value', 'CONF_STATUS', 'Confidentiality status (flag)']
Categorical columns used: ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'c_resid', 'Country of residence', 'unit', 'Unit of measure', 'nace_r2', 'Statistical classification of economic activities in the European Community (NACE Rev. 2)', 'geo', 'Geopolitical entity (reporting)', 'TIME_PERIOD', 'OBS_FLAG', 'Observation status (Flag) V2 structure']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,OBS_VALUE,385,385,0,2.529180e+07,7.068401e+07,11228.0,820739.0,2641896.0,10736324.0,500449230.0,9915585.0,-14052638.5,25609701.5,58.0
1,CONF_STATUS,385,0,385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Confidentiality status (flag),385,0,385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Observation value,385,0,385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Time,385,0,385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (reporting),385,385,0,39,Albania,10
1,geo,385,385,0,39,AL,10
2,TIME_PERIOD,385,385,0,10,2024-11,39
3,OBS_FLAG,385,36,349,2,e,31
4,Observation status (Flag) V2 structure,385,36,349,2,estimated,31
5,Country of residence,385,385,0,1,Total,385
6,STRUCTURE,385,385,0,1,dataflow,385
7,STRUCTURE_ID,385,385,0,1,ESTAT:TOUR_OCC_NIM(1.0),385
8,STRUCTURE_NAME,385,385,0,1,Nights spent at tourist accommodation establis...,385
9,Statistical classification of economic activit...,385,385,0,1,Hotels; holiday and other short-stay accommoda...,385


In [ ]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "trng_lfse_01_page_linear_2_0.csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: Participation rate in education and training (last 4 weeks), 25 to 74 years
# TIME_PERIOD: yearly 

Analyzing: trng_lfse_01_page_linear_2_0.csv
Raw dataframe shape: (361, 21) Processed shape: (361, 43)
Numeric columns used: ['TIME_PERIOD', 'Time', 'OBS_VALUE', 'Observation value', 'CONF_STATUS', 'Confidentiality status (flag)']
Categorical columns used: ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'unit', 'Unit of measure', 'sex', 'Sex', 'age', 'Age class', 'geo', 'Geopolitical entity (reporting)', 'OBS_FLAG', 'Observation status (Flag) V2 structure']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,OBS_VALUE,361,361,0,10.990582,7.631369,0.8,5.2,9.1,15.6,35.6,10.4,-10.4,31.2,4.0
1,TIME_PERIOD,361,361,0,2019.432133,2.871598,2015.0,2017.0,2019.0,2022.0,2024.0,5.0,2009.5,2029.5,0.0
2,CONF_STATUS,361,0,361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Confidentiality status (flag),361,0,361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Observation value,361,0,361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Time,361,0,361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (reporting),361,361,0,38,Austria,10
1,geo,361,361,0,38,AT,10
2,OBS_FLAG,361,57,304,3,b,55
3,Observation status (Flag) V2 structure,361,57,304,3,break in time series,55
4,Age class,361,361,0,1,From 25 to 74 years,361
5,STRUCTURE,361,361,0,1,dataflow,361
6,STRUCTURE_ID,361,361,0,1,ESTAT:TRNG_LFSE_01(1.0),361
7,STRUCTURE_NAME,361,361,0,1,Participation rate in education and training (...,361
8,Sex,361,361,0,1,Total,361
9,Time frequency,361,361,0,1,Annual,361


In [ ]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "urb_ceduc_page_linear_2_0.csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: Students in higher education (ISCED level 5-8 from 2014 onwards), total
# TIME_PERIOD: yearly 
# CITY BASED, with county sums (filter out when analysing)


Analyzing: urb_ceduc_page_linear_2_0.csv
Raw dataframe shape: (4951, 17) Processed shape: (4951, 35)
Numeric columns used: ['TIME_PERIOD', 'Time', 'OBS_VALUE', 'Observation value', 'CONF_STATUS', 'Confidentiality status (flag)']
Categorical columns used: ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'indic_ur', 'Urban audit indicator', 'cities', 'Geopolitical entity (declaring)', 'OBS_FLAG', 'Observation status (Flag) V2 structure']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,OBS_VALUE,4951,4951,0,33435.531206,161342.273294,0.0,2464.5,8961.0,24544.5,2979313.0,22080.0,-30655.5,57664.5,465.0
1,TIME_PERIOD,4951,4951,0,2018.644718,2.475009,2015.0,2017.0,2018.0,2021.0,2024.0,4.0,2011.0,2027.0,0.0
2,CONF_STATUS,4951,0,4951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Confidentiality status (flag),4951,0,4951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Observation value,4951,0,4951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Time,4951,0,4951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (declaring),4951,4951,0,741,Alytus,10
1,cities,4951,4951,0,741,EE001C,10
2,OBS_FLAG,4951,224,4727,4,d,145
3,Observation status (Flag) V2 structure,4951,224,4727,4,definition differs (see metadata),145
4,STRUCTURE,4951,4951,0,1,dataflow,4951
5,STRUCTURE_ID,4951,4951,0,1,ESTAT:URB_CEDUC(1.0),4951
6,STRUCTURE_NAME,4951,4951,0,1,Education - cities and greater cities,4951
7,Time frequency,4951,4951,0,1,Annual,4951
8,Urban audit indicator,4951,4951,0,1,Students in higher education (ISCED level 5-8 ...,4951
9,freq,4951,4951,0,1,A,4951


In [25]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "urb_clma_page_linear_2_0.csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: Labour market, Persons unemployed, total
# TIME_PERIOD: yearly 
# CITY BASED, with county sums (filter out when analysing)

Analyzing: urb_clma_page_linear_2_0.csv
Raw dataframe shape: (4892, 17) Processed shape: (4892, 35)
Numeric columns used: ['TIME_PERIOD', 'Time', 'OBS_VALUE', 'Observation value', 'CONF_STATUS', 'Confidentiality status (flag)']
Categorical columns used: ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'indic_ur', 'Urban audit indicator', 'cities', 'Geopolitical entity (declaring)', 'OBS_FLAG', 'Observation status (Flag) V2 structure']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,TIME_PERIOD,4892,4892,0,2018.662919,2.418796,2015.0,2017.0,2019.0,2021.0,2024.0,4.0,2011.0,2027.0,0.0
1,OBS_VALUE,4892,4864,28,29839.072527,236028.451439,1.0,2703.0,5500.0,11211.0,5055988.0,8508.0,-10059.0,23973.0,458.0
2,CONF_STATUS,4892,0,4892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Confidentiality status (flag),4892,0,4892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Observation value,4892,0,4892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Time,4892,0,4892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (declaring),4892,4892,0,782,Alytus,10
1,cities,4892,4892,0,782,BE,10
2,OBS_FLAG,4892,2577,2315,5,e,1583
3,Observation status (Flag) V2 structure,4892,2577,2315,5,estimated,1583
4,STRUCTURE,4892,4892,0,1,dataflow,4892
5,STRUCTURE_ID,4892,4892,0,1,ESTAT:URB_CLMA(1.0),4892
6,STRUCTURE_NAME,4892,4892,0,1,Labour market - cities and greater cities,4892
7,Time frequency,4892,4892,0,1,Annual,4892
8,Urban audit indicator,4892,4892,0,1,"Persons unemployed, total",4892
9,freq,4892,4892,0,1,A,4892


In [26]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "urb_clma_page_linear_2_0(1).csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: Persons employed, 20-64, total
# TIME_PERIOD: yearly 
# CITY BASED, with county sums (filter out when analysing)

Analyzing: urb_clma_page_linear_2_0(1).csv
Raw dataframe shape: (4631, 17) Processed shape: (4631, 35)
Numeric columns used: ['TIME_PERIOD', 'Time', 'OBS_VALUE', 'Observation value', 'CONF_STATUS', 'Confidentiality status (flag)']
Categorical columns used: ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'indic_ur', 'Urban audit indicator', 'cities', 'Geopolitical entity (declaring)', 'OBS_FLAG', 'Observation status (Flag) V2 structure']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,TIME_PERIOD,4631,4631,0,2018.713669,2.513906e+00,2015.0,2017.0,2019.0,2021.0,2024.0,4.0,2011.00,2027.00,0.0
1,OBS_VALUE,4631,4627,4,273735.419192,1.898800e+06,10662.0,32822.5,53509.0,100758.0,39533380.0,67935.5,-69080.75,202661.25,506.0
2,CONF_STATUS,4631,0,4631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Confidentiality status (flag),4631,0,4631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Observation value,4631,0,4631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Time,4631,0,4631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (declaring),4631,4631,0,718,Alytus,10
1,cities,4631,4631,0,718,EE001C,10
2,OBS_FLAG,4631,2802,1829,4,e,1810
3,Observation status (Flag) V2 structure,4631,2802,1829,4,estimated,1810
4,STRUCTURE,4631,4631,0,1,dataflow,4631
5,STRUCTURE_ID,4631,4631,0,1,ESTAT:URB_CLMA(1.0),4631
6,STRUCTURE_NAME,4631,4631,0,1,Labour market - cities and greater cities,4631
7,Time frequency,4631,4631,0,1,Annual,4631
8,Urban audit indicator,4631,4631,0,1,"Persons employed, 20-64, total",4631
9,freq,4631,4631,0,1,A,4631


In [27]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "urb_cpop1_page_linear_2_0.csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: Population on 1 January by age groups and sex - cities and greater cities, total
# TIME_PERIOD: yearly 
# Cities


Analyzing: urb_cpop1_page_linear_2_0.csv
Raw dataframe shape: (6370, 17) Processed shape: (6370, 35)
Numeric columns used: ['TIME_PERIOD', 'Time', 'OBS_VALUE', 'Observation value', 'CONF_STATUS', 'Confidentiality status (flag)']
Categorical columns used: ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'indic_ur', 'Urban audit indicator', 'cities', 'Geopolitical entity (declaring)', 'OBS_FLAG', 'Observation status (Flag) V2 structure']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,OBS_VALUE,6370,6370,0,692015.032182,4.729413e+06,5506.0,77711.0,121387.5,221387.75,83155031.0,143676.75,-137804.125,436902.875,708.0
1,TIME_PERIOD,6370,6370,0,2018.935950,2.672818e+00,2015.0,2017.0,2019.0,2021.00,2024.0,4.00,2011.000,2027.000,0.0
2,CONF_STATUS,6370,0,6370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Confidentiality status (flag),6370,0,6370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Observation value,6370,0,6370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Time,6370,0,6370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (declaring),6370,6370,0,881,Aachen,10
1,cities,6370,6370,0,881,BE,10
2,OBS_FLAG,6370,1000,5370,3,b,424
3,Observation status (Flag) V2 structure,6370,1000,5370,3,break in time series,424
4,STRUCTURE,6370,6370,0,1,dataflow,6370
5,STRUCTURE_ID,6370,6370,0,1,ESTAT:URB_CPOP1(1.0),6370
6,STRUCTURE_NAME,6370,6370,0,1,Population on 1 January by age groups and sex ...,6370
7,Time frequency,6370,6370,0,1,Annual,6370
8,Urban audit indicator,6370,6370,0,1,"Population on the 1st of January, total",6370
9,freq,6370,6370,0,1,A,6370


In [ ]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "urb_cpopcb_page_linear_2_0.csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: Population by citizenship and country of birth - cities and greater cities
# TIME_PERIOD: yearly 
# city based, with county sums (filter out when analysing)
# Foreigners 

Analyzing: urb_cpopcb_page_linear_2_0.csv
Raw dataframe shape: (4245, 17) Processed shape: (4245, 35)
Numeric columns used: ['TIME_PERIOD', 'Time', 'OBS_VALUE', 'Observation value', 'CONF_STATUS', 'Confidentiality status (flag)']
Categorical columns used: ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'indic_ur', 'Urban audit indicator', 'cities', 'Geopolitical entity (declaring)', 'OBS_FLAG', 'Observation status (Flag) V2 structure']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,OBS_VALUE,4245,4245,0,86777.583274,477233.29107,617.0,9845.0,19456.0,39086.0,8703607.0,29241.0,-34016.5,82947.5,535.0
1,TIME_PERIOD,4245,4245,0,2018.877503,2.73432,2015.0,2016.0,2019.0,2021.0,2024.0,5.0,2008.5,2028.5,0.0
2,CONF_STATUS,4245,0,4245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Confidentiality status (flag),4245,0,4245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Observation value,4245,0,4245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Time,4245,0,4245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (declaring),4245,4245,0,705,Antwerpen (greater city),10
1,cities,4245,4245,0,705,BE,10
2,OBS_FLAG,4245,834,3411,2,e,503
3,Observation status (Flag) V2 structure,4245,834,3411,2,estimated,503
4,STRUCTURE,4245,4245,0,1,dataflow,4245
5,STRUCTURE_ID,4245,4245,0,1,ESTAT:URB_CPOPCB(1.0),4245
6,STRUCTURE_NAME,4245,4245,0,1,Population by citizenship and country of birth...,4245
7,Time frequency,4245,4245,0,1,Annual,4245
8,Urban audit indicator,4245,4245,0,1,Foreign-born,4245
9,freq,4245,4245,0,1,A,4245


In [29]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "urb_ctour_page_linear_2_0.csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: Total nights spent in tourist accommodation establishments
# TIME_PERIOD: yearly 
# city based, with county sums (filter out when analysing)


Analyzing: urb_ctour_page_linear_2_0.csv
Raw dataframe shape: (3864, 17) Processed shape: (3864, 35)
Numeric columns used: ['TIME_PERIOD', 'Time', 'OBS_VALUE', 'Observation value']
Categorical columns used: ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'indic_ur', 'Urban audit indicator', 'cities', 'Geopolitical entity (declaring)', 'OBS_FLAG', 'Observation status (Flag) V2 structure', 'CONF_STATUS', 'Confidentiality status (flag)']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,TIME_PERIOD,3864,3864,0,2.018141e+03,2.269872e+00,2015.0,2016.0,2018.0,2020.0,2024.0,4.0,2010.0,2026.0,0.0
1,OBS_VALUE,3864,3689,175,4.358833e+06,3.403005e+07,0.0,143900.0,367586.0,907999.0,468476741.0,764099.0,-1002248.5,2054147.5,474.0
2,Observation value,3864,0,3864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Time,3864,0,3864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (declaring),3864,3864,0,633,Acireale (greater city),10
1,cities,3864,3864,0,633,IT,10
2,OBS_FLAG,3864,766,3098,3,e,598
3,Observation status (Flag) V2 structure,3864,766,3098,3,estimated,598
4,CONF_STATUS,3864,175,3689,1,C,175
5,Confidentiality status (flag),3864,175,3689,1,confidential,175
6,STRUCTURE,3864,3864,0,1,dataflow,3864
7,STRUCTURE_ID,3864,3864,0,1,ESTAT:URB_CTOUR(1.0),3864
8,STRUCTURE_NAME,3864,3864,0,1,Culture and tourism - cities and greater cities,3864
9,Time frequency,3864,3864,0,1,Annual,3864


In [30]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "urb_ctran_page_linear_2_0.csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: Transport, Number of registered cars per 1000 population
# TIME_PERIOD: yearly 
# city based, with county sums (filter out when analysing)

Analyzing: urb_ctran_page_linear_2_0.csv
Raw dataframe shape: (5084, 17) Processed shape: (5084, 35)
Numeric columns used: ['TIME_PERIOD', 'Time', 'OBS_VALUE', 'Observation value', 'CONF_STATUS', 'Confidentiality status (flag)']
Categorical columns used: ['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'freq', 'Time frequency', 'indic_ur', 'Urban audit indicator', 'cities', 'Geopolitical entity (declaring)', 'OBS_FLAG', 'Observation status (Flag) V2 structure']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,TIME_PERIOD,5084,5084,0,2018.685287,2.516460,2015.00,2017.00,2018.00,2021.00,2024.00,4.00,2011.000,2027.000,0.0
1,OBS_VALUE,5084,5081,3,491.412854,185.892087,37.62,415.88,465.17,534.93,4994.89,119.05,237.305,713.505,169.0
2,CONF_STATUS,5084,0,5084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Confidentiality status (flag),5084,0,5084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Observation value,5084,0,5084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Time,5084,0,5084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,Geopolitical entity (declaring),5084,5084,0,741,Acireale (greater city),10
1,cities,5084,5084,0,741,BE,10
2,OBS_FLAG,5084,617,4467,4,b,437
3,Observation status (Flag) V2 structure,5084,617,4467,4,break in time series,437
4,STRUCTURE,5084,5084,0,1,dataflow,5084
5,STRUCTURE_ID,5084,5084,0,1,ESTAT:URB_CTRAN(1.0),5084
6,STRUCTURE_NAME,5084,5084,0,1,Transport - cities and greater cities,5084
7,Time frequency,5084,5084,0,1,Annual,5084
8,Urban audit indicator,5084,5084,0,1,Number of registered cars per 1000 population,5084
9,freq,5084,5084,0,1,A,5084


In [31]:
from IPython.display import display

# Template for per-CSV analysis — copy this cell and edit values
CSV_TO_ANALYZE = os.path.join(RAW_DIR, "who_ambient_air_quality_database_version_2024_(v6.1).csv")
NUMERIC_COLUMNS = None  # e.g., ["NUM_COL_1", "NUM_COL_2"]
CATEGORICAL_COLUMNS = None  # e.g., ["CAT_COL_1", "CAT_COL_2"]

print(f"Analyzing: {os.path.basename(CSV_TO_ANALYZE)}")
df = pd.read_csv(CSV_TO_ANALYZE, low_memory=False)
processed = add_derived_columns(df) # , numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)
numeric_df, categorical_df = profile_dataframe(df)#, numeric_include=NUMERIC_COLUMNS, categorical_include=CATEGORICAL_COLUMNS)

print("Raw dataframe shape:", df.shape, "Processed shape:", processed.shape)
print("Numeric columns used:", [] if numeric_df.empty else list(numeric_df["column"]))
print("Categorical columns used:", [] if categorical_df.empty else list(categorical_df["column"]))

# display(processed.head(10))
display(numeric_df.sort_values(["non_null_count", "column"], ascending=[False, True]).reset_index(drop=True))
display(categorical_df.sort_values(["distinct_count", "column"], ascending=[False, True]).reset_index(drop=True))

# Relevant columns
# OBS_VALUE: Air quailty data 
# TIME_PERIOD: yearly 
# city based

Analyzing: who_ambient_air_quality_database_version_2024_(v6.1).csv
Raw dataframe shape: (26801, 20) Processed shape: (26801, 40)
Numeric columns used: ['year', 'pm10_concentration', 'pm25_concentration', 'no2_concentration', 'pm10_tempcov', 'pm25_tempcov', 'no2_tempcov', 'population', 'latitude', 'longitude', 'who_ms']
Categorical columns used: ['who_region', 'iso3', 'country_name', 'city', 'version', 'type_of_stations', 'reference', 'web_link', 'population_source']


,column,row_count,non_null_count,null_count,mean,stddev,min,q1,median,q3,max,iqr,lower_bound,upper_bound,outlier_count
0,latitude,26801,26801,0,46.299143,7.428719,-21.33866,42.5806,46.821100,50.62588,6.967750e+01,8.04528,30.51268,62.69380,739
1,longitude,26801,26801,0,10.584859,11.773289,-63.08150,4.0900,10.299445,16.67530,7.458330e+01,12.58530,-14.78795,35.55325,837
2,who_ms,26801,26801,0,1.000000,0.000000,1.00000,1.0000,1.000000,1.00000,1.000000e+00,0.00000,1.00000,1.00000,0
3,year,26801,26801,0,2016.326368,3.177053,2010.00000,2014.0000,2017.000000,2019.00000,2.022000e+03,5.00000,2006.50000,2026.50000,0
4,no2_concentration,26801,22638,4163,18.884829,26.767195,0.01600,10.4555,17.033500,24.96850,3.670314e+03,14.51300,-11.31400,46.73800,432
5,pm10_concentration,26801,22281,4520,22.796307,11.613333,0.73000,15.8160,20.222000,26.33100,1.466950e+02,10.51500,0.04350,42.10350,1291
6,no2_tempcov,26801,21808,4993,92.329008,13.545350,0.00000,93.0000,96.000000,99.00000,1.000000e+02,6.00000,84.00000,108.00000,2132
7,pm10_tempcov,26801,20815,5986,90.207975,17.731059,0.00000,91.0000,97.000000,99.00000,1.000000e+02,8.00000,79.00000,111.00000,2427
8,pm25_concentration,26801,11577,15224,13.613745,6.569392,1.00000,9.2700,12.329000,16.60500,8.500000e+01,7.33500,-1.73250,27.60750,359
9,pm25_tempcov,26801,11031,15770,87.427885,21.095033,0.00000,89.0000,96.000000,99.00000,1.000000e+02,10.00000,74.00000,114.00000,1512


,column,row_count,non_null_count,null_count,distinct_count,mode,mode_freq
0,city,26801,26801,0,4347,Albacete/ESP,12
1,type_of_stations,26801,22840,3961,288,Urban,7580
2,population_source,26801,6149,20652,131,"old database, old database",904
3,version,26801,26801,0,91,V6.0 (2023),15916
4,country_name,26801,26801,0,49,Italy,4035
5,iso3,26801,26801,0,49,ITA,4035
6,reference,26801,1404,25397,22,European Environmental Agency,894
7,web_link,26801,1281,25520,16,https://www.eea.europa.eu/themes/air/explore-a...,893
8,who_region,26801,26801,0,1,4_Eur,26801
